# *Data load*

In [1]:
import config
from pprint import pprint
from pshmodule.utils import filemanager as fm

In [2]:
print("data load")

data = []
with open(config.train_json, 'r', encoding='utf-8') as f:
    for line in f:
        data.append(json.loads(line.rstrip('\n|\r')))
df = pd.DataFrame(data)

print(df.head())

NameError: name 'data' is not defined

In [3]:
df.head()

,content,label
0,오늘 반장 선거에서 내가 반장이 됐어 친구들이 날 믿어줘서 너무 고맙다 친구들에게 ...,1
1,오늘 시험에 친구가 알려준 문제가 나왔어 결과가 좋아서 기쁘다 말로 하기 쑥스러운데...,1
2,오늘 대학 합격 통지를 받았어 엄마가 너무 기뻐하는데 나는 눈물이 났어 사실 이렇게...,1
3,오늘 졸업식인데 선생님께서 아프셔서 오지 못하셨어 그동안 감사하다는 말도 못했는데 ...,1
4,나 올림피아드에서 금상 받았어 그동안 고생이 헛되지 않아서 너무 기뻐 근데 친구들이...,1


# *Definition*

##### 사전 크기, 사용자 정의 토큰 정의

In [4]:
vocab_size = 24000

In [5]:
user_defined_symbols = ["<pad>", "<unk>", "<cls>", "<sep>", "<mask>", "<bos>", "<eos>", "<tsep>", "<name>", "<url>"]
user_defined_symbols += ["<unk0>", "<unk1>", "<unk2>", "<unk3>", "<unk4>", "<unk5>", "<unk6>", "<unk7>", "<unk8>", "<unk9>"]

In [6]:
unused_token_num = 100
unused_list = [f"<unused{i}>" for i in range(unused_token_num)]
user_defined_symbols += unused_list

In [7]:
pprint(user_defined_symbols)

['<pad>',
 '<unk>',
 '<cls>',
 '<sep>',
 '<mask>',
 '<bos>',
 '<eos>',
 '<tsep>',
 '<name>',
 '<url>',
 '<unk0>',
 '<unk1>',
 '<unk2>',
 '<unk3>',
 '<unk4>',
 '<unk5>',
 '<unk6>',
 '<unk7>',
 '<unk8>',
 '<unk9>',
 '<unused0>',
 '<unused1>',
 '<unused2>',
 '<unused3>',
 '<unused4>',
 '<unused5>',
 '<unused6>',
 '<unused7>',
 '<unused8>',
 '<unused9>',
 '<unused10>',
 '<unused11>',
 '<unused12>',
 '<unused13>',
 '<unused14>',
 '<unused15>',
 '<unused16>',
 '<unused17>',
 '<unused18>',
 '<unused19>',
 '<unused20>',
 '<unused21>',
 '<unused22>',
 '<unused23>',
 '<unused24>',
 '<unused25>',
 '<unused26>',
 '<unused27>',
 '<unused28>',
 '<unused29>',
 '<unused30>',
 '<unused31>',
 '<unused32>',
 '<unused33>',
 '<unused34>',
 '<unused35>',
 '<unused36>',
 '<unused37>',
 '<unused38>',
 '<unused39>',
 '<unused40>',
 '<unused41>',
 '<unused42>',
 '<unused43>',
 '<unused44>',
 '<unused45>',
 '<unused46>',
 '<unused47>',
 '<unused48>',
 '<unused49>',
 '<unused50>',
 '<unused51>',
 '<unused52>',
 '

# *Tokenizer Train*

##### Huggingface BPE Tokenizer

In [8]:
from tokenizers import Tokenizer, models, normalizers, pre_tokenizers, decoders, trainers
from transformers import GPT2TokenizerFast, AutoTokenizer

In [9]:
tokenizer = Tokenizer(models.BPE())
print(f"tokenizer : {tokenizer}")

tokenizer : <tokenizers.Tokenizer object at 0x88b0c90>


In [10]:
tokenizer.normalizer = normalizers.NFKC()
tokenizer.pre_tokenizer = pre_tokenizers.Sequence([pre_tokenizers.Metaspace()])
tokenizer.decoders = decoders.Metaspace()

In [11]:
trainer = trainers.BpeTrainer(
    vocab_size=vocab_size, 
    show_progress=True,
    special_tokens=user_defined_symbols,
)

In [12]:
def gen():
    for row in df.content:
        yield row

In [13]:
tokenizer.train_from_iterator(gen(), trainer=trainer)

In [14]:
tokenizer.model.save('../../tokenizer/temp')

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
rm: cannot remove '../../tokenizer/temps': No such file or directory


['../../tokenizer/temp/vocab.json', '../../tokenizer/temp/merges.txt']

##### test

In [15]:
output = tokenizer.encode("본 고안은 이러한 특성을 이용해 사용한다.")
print(output.ids)

[2107, 1754, 10967, 4617, 12549, 4439, 1885, 1876]


In [16]:
tokenizer.decode(output.ids)

'▁본 ▁고 안은 ▁이러한 ▁특성을 ▁이용해 ▁사용 한다.'

In [17]:
tokenizer.decoder = decoders.BPEDecoder(suffix='_')
tokenizer.decode(output.ids)

'▁본▁고안은▁이러한▁특성을▁이용해▁사용한다.'

# *Save according to form*

In [18]:
tokenizer_for_load = GPT2TokenizerFast.from_pretrained("../../tokenizer/temp")  # 로드

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


##### Definition

In [19]:
tokenizer_for_load.pad_token = "<pad>"
tokenizer_for_load.unk_token = "<unk>"
tokenizer_for_load.cls_token = "<cls>"
tokenizer_for_load.sep_token = "<sep>"
tokenizer_for_load.mask_token = "<mask>"
tokenizer_for_load.bos_token = "<bos>"
tokenizer_for_load.eos_token = "<eos>"

special_tokens_dict = {'additional_special_tokens': user_defined_symbols}
tokenizer_for_load.add_special_tokens(special_tokens_dict)

0

In [20]:
tokenizer_for_load.save_pretrained("../../tokenizer", legacy_format=False)

('../../tokenizer/tokenizer_config.json',
 '../../tokenizer/special_tokens_map.json',
 '../../tokenizer/tokenizer.json')

in `tokenizer.json`  
```json
    "normalizer": {
        "type": "Sequence",
        "normalizers": [
            {
                "type": "NFKC"
            },
            {
                "type": "BertNormalizer",
                "clean_text": false,
                "handle_chinese_chars": false,
                "strip_accents": false,
                "lowercase": false
            }
        ]
    },
    "pre_tokenizer": {
        "type": "Sequence",
        "pretokenizers": [
            {
                "type": "Metaspace",
                "replacement": "▁",
                "add_prefix_space": true
            }
        ]
    },
    "post_processor": null,
    "decoder": {
        "type": "Metaspace",
        "replacement": "▁",
        "add_prefix_space": true
    },
```

in `tokenizer_config.json`  
```json
,
    "model_type": "gpt2"
```

rename `tokenizer_config.json` => `config.json`

# *Test*

In [21]:
t = AutoTokenizer.from_pretrained("../../tokenizer")

In [22]:
e = t("본 고안은 이러한 특성을 이용해 사용한다.")
print(e)
print(t.decode(e['input_ids']))

{'input_ids': [2107, 1754, 10967, 4617, 12549, 4439, 1885, 1876], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1]}
본 고안은 이러한 특성을 이용해 사용한다.
